In [1]:
# !pip3 install tensorflow==2.4.1

In [ ]:
import tensorflow_hub as tfhub
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

In [ ]:
import numpy as np

wav_as_float_or_int16 = np.sin(np.linspace(-np.pi, np.pi, 128), dtype=np.float32)

In [ ]:
model = tf.saved_model.load('/Users/huseinzolkepli/Downloads/nonsemantic-speech-benchmark_trill_3')

In [ ]:
X = tf.compat.v1.placeholder(tf.float32, [None, None])
o = model(samples = X, sample_rate = 16000)
o

In [ ]:
logits = tf.identity(o['layer19'], name = 'logits')
embedding = tf.identity(o['embedding'], name = 'embedding')
# o = tf.identity(o, name = 'logits')
# o.shape

In [ ]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [ ]:
p = sess.run([logits, embedding], feed_dict = {X: [wav_as_float_or_int16, wav_as_float_or_int16]})

In [ ]:
p[0].shape, p[1].shape

In [ ]:
saver = tf.compat.v1.train.Saver()
saver.save(sess, 'out/model.ckpt')

In [ ]:
strings = ','.join(
    [
        n.name
        for n in tf.compat.v1.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'top_p' in n.name
        or 'temperature' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'embedding' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'gradients' not in n.name
        and 'save/AssignVariableOp' not in n.name
        and not n.name.startswith('AssignVariableOp')
    ]
)
strings.split(',')

In [ ]:
def freeze_graph(model_dir, output_node_names):

    if not tf.compat.v1.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.compat.v1.Session(graph = tf.compat.v1.Graph()) as sess:
        saver = tf.compat.v1.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.compat.v1.graph_util.convert_variables_to_constants(
            sess,
            tf.compat.v1.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.compat.v1.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [ ]:
freeze_graph('out', strings)

In [ ]:
with tf.compat.v1.gfile.GFile('out/frozen_model.pb', 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

In [ ]:
for n in graph_def.node:
    if 'network/layer1/conv/weights' in n.name:
        break

In [ ]:
n

In [ ]:
with tf.compat.v1.gfile.GFile('out/frozen_model.pb', 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    
for n in graph_def.node:
    if 'StatefulPartitionedCall' in n.name:
        break
        
for i in range(len(n.attr['Tin'].list.type)):
    if n.attr['Tin'].list.type[i] == 20:
        n.attr['Tin'].list.type[i] = 1
        
with tf.compat.v1.gfile.GFile('out/frozen_model.pb', 'wb') as f:
    f.write(graph_def.SerializeToString())

In [ ]:
n

In [ ]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.compat.v1.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in range(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.compat.v1.Graph().as_default() as graph:
        tf.compat.v1.import_graph_def(graph_def)
    return graph

In [ ]:
g = load_graph('out/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
logits = g.get_tensor_by_name('import/logits:0')
embedding = g.get_tensor_by_name('import/embedding:0')
test_sess = tf.compat.v1.Session(graph = g)

In [ ]:
test_sess.run(embedding, feed_dict = {x: [wav_as_float_or_int16, wav_as_float_or_int16]})